# 训练数据生成
进行内部数据转换，生成模型微调所需训练和测试数据

### 读入数据

In [2]:
import json
import os


# 从 JSON 文件中读取数据
input_file_path = 'data2text/data2text-test-all-v1.json'

with open(input_file_path, 'r', encoding='utf-8') as file:
    input_data = json.load(file)

output_data = []
output_data_test = []
i = 0

def retain_entity(value):
    if value == "" or ("未提及" in value) or ("未明确" in value) or ("未提供" in value):
        return False
    else:
        return True

In [ ]:
# 将“装备”中的信息划分并加入“装备名称”和装备数量中
# 遍历输入数据中的每个data项
for excel in input_data:
    for sheet in excel:
        entities = []

        # 遍历data中的每一行
        for row, row_value in sheet['table']['data'].items():
            for value in row_value:
                header_index = row_value.index(value)
                entity_type = sheet['table']['header'][header_index]
                if entity_type == "装备":
                    zb_name = []
                    zb_number = []
                    if '；' not in value:
                        # 将装备直接加入装备名称
                        if "装备名称" not in sheet['table']['header']:
                            sheet['table']['header'].append("装备名称")
                        row_value.append(value)
                    if '；' in value:
                        if "装备名称" not in sheet['tabel']['header']:
                            sheet['table']['header'].append("装备名称")
                        if "装备数量" not in sheet['tabel']['header']:
                            sheet['table']['header'].append("装备数量")
                        # 将装备切分后加入装备名称和装备数量
                        sheet['table']['header'].append("装备数量")
                        sub_values = value.split('；')
                        for zhuangbei in sub_values:
                            zb = zhaungbei.split('；')
                            zb_name.append(zb[0])
                            zb_number.append(zb[1])
                            zb_number_set = list(set(zb_number))
            row_value.append()


                if entity_type not in ["国家", "人物", "任务", "行动", "事件"]:
                    if value != "" and "未提及" not in value and "未明确" not in value and "未提供" not in value:
                        entities.append({
                            "entity": value,
                            "entity_type": entity_type})

### header出现次数

In [21]:
entity_types = set([])
for data in input_data:
    entities = []
    i += 1
   
    if isinstance(data, list):
        for row in data:
            if isinstance(row, dict):
                for header in row['table']['header']:
                    entity_types.add(header)
                            

    # # 遍历data中的每一行
    # for row in data['table']['data']:
    #     for item in row:
    #         header_index = row.index(item)
    #         entity_type = data['table']['header'][header_index]
    #         entity_types.add(entity_type)

# 打印集合内容
print("不同的 entity_type 有：", entity_types)

# 打印总共有多少种不同的 entity_type
print("共有", len(entity_types), "种不同的 entity_type")

不同的 entity_type 有： {'组织机构', '任务类型', '备注', '指挥员', '主体数量', '主体名称', '主体类型', '行动', '军种', '数量', '任务状态', '时间', '地点', '人物', '开始时间', '装备名称', '装备', '单位', '人数', '装备类型', '事件', '国家', '任务', '国别', '装备数量'}
共有 25 种不同的 entity_type


### 生成训练集和测试集

In [12]:
import numpy as np

train_val_set = []
test_set = []
all_set = []

# 遍历输入数据中的每个data项
for excel in input_data:
    for sheet in excel:
        entities = []

        # 遍历data中的每一行
        for row, row_value in sheet['table']['data'].items():
            for value in row_value:
                header_index = row_value.index(value)
                entity_type = sheet['table']['header'][header_index]
                if entity_type not in ["国家", "人物", "任务", "行动", "事件"]:
                    if value != "" and "未提及" not in value and "未明确" not in value and "未提供" not in value:
                        entities.append({
                            "entity": value,
                            "entity_type": entity_type})
                        if entity_type in ["装备"]:
                            if "；" in value:
                                zb_name = []
                                zb_number = []
                                sub_values = value.split('；')
                                for zhuangbei in sub_values:
                                    zb = zhuangbei.split('，')
                                    zb_name.append(zb[0])
                                    zb_number.append(zb[1])
                                    zb_number_set = list(set(zb_number))
                                for zb_na in zb_name:
                                    entities.append({
                                        "entity": zb_na,
                                        "entity_type": "装备名称"})
                                for zb_nu in zb_number_set:
                                    entities.append({
                                        "entity": zb_nu,
                                        "entity_type": "装备数量"})
                            else:
                                entities.append({
                                    "entity": value,
                                    "entity_type": "装备名称"})

        # 创建输出格式
        output = {"text": sheet["text"], "entity": entities}
        all_set.append(output)

# 打乱数组顺序
np.random.shuffle(all_set)

# 计算分割点
split_index = int(len(all_set) * 0.9)
print(len(all_set))

# 划分训练集和测试集
train_val_set = all_set[:split_index]
test_set = all_set[split_index:]

# 获取当前工作目录
current_path = os.getcwd()
output_file_path = os.path.join(current_path, 'data2text/data2text-ner-train-val.json')
test_file_path = os.path.join(current_path, 'data2text/data2text-ner-test.json')

# 打开输出路径
with open(output_file_path, 'w', encoding='utf-8') as file1:
    json.dump(train_val_set, file1, ensure_ascii=False, indent=1)
with open(test_file_path, 'w', encoding='utf-8') as file2:
    json.dump(test_set, file2, ensure_ascii=False, indent=1)

2148


## 转化测试集

In [8]:
import numpy as np

train_val_set = []
test_set = []
all_set = []

# 遍历输入数据中的每个data项
for excel in input_data:
    for sheet in excel:
        entities = []

        # 遍历data中的每一行
        for row, row_value in sheet['table']['data'].items():
            for value in row_value:
                header_index = row_value.index(value)
                entity_type = sheet['table']['header'][header_index]
                if entity_type not in ["任务", "行动", "事件"]:
                    if value != "" and "未提及" not in value and "未明确" not in value and "未提供" not in value:
                        entities.append({
                            "entity": value,
                            "entity_type": entity_type})

        # 创建输出格式
        output = {"text": sheet["text"], "entity": entities}
        all_set.append(output)

# 打乱数组顺序
np.random.shuffle(all_set)

# 计算分割点
split_index = int(len(all_set) * 0.9)
print(len(all_set))

# 划分训练集和测试集
train_val_set = all_set[:split_index]
test_set = all_set[split_index:]

# 获取当前工作目录
current_path = os.getcwd()
test_file_path = os.path.join(current_path, 'data2text/iepile-ner-test-v2.json')

# 打开输出路径
with open(test_file_path, 'w', encoding='utf-8') as file2:
    json.dump(test_set, file2, ensure_ascii=False, indent=1)

2148


## 转变为jsonl格式

In [13]:
import json
import jsonlines
import os


def rename_file(old_name, new_name):
    # 确保旧文件存在
    if os.path.exists(old_name):
        # 重命名文件
        os.rename(old_name, new_name)
        print(f"File renamed from {old_name} to {new_name}")
    else:
        print(f"File {old_name} does not exist.")


train_json_file = "data2text/data2text-ner-train-val.json"
train_jsonl_file = "data2text/data2text-ner-train-val.jsonl"
test_json_file = "data2text/data2text-ner-test.json"
test_jsonl_file = "data2text/data2text-ner-test.jsonl"

with open(train_json_file, "r", encoding='utf-8') as file:
    with jsonlines.open(train_jsonl_file, 'w') as writer:
        input_data = json.load(file)
        for data in input_data:
            writer.write(data)

with open(test_json_file, "r", encoding='utf-8') as file:
    with jsonlines.open(test_jsonl_file, 'w') as writer:
        input_data = json.load(file)
        for data in input_data:
            writer.write(data)

rename_file("data2text/data2text-ner-train-val.jsonl", "data2text/data2text-ner-train-val-json.json")
rename_file("data2text/data2text-ner-test.jsonl", "data2text/data2text-ner-test-json.json")


File renamed from data2text/data2text-ner-train-val.jsonl to data2text/data2text-ner-train-val-json.json
File renamed from data2text/data2text-ner-test.jsonl to data2text/data2text-ner-test-json.json


## 转化测试集

In [9]:
import json
import jsonlines
import os


def rename_file(old_name, new_name):
    # 确保旧文件存在
    if os.path.exists(old_name):
        # 重命名文件
        os.rename(old_name, new_name)
        print(f"File renamed from {old_name} to {new_name}")
    else:
        print(f"File {old_name} does not exist.")


test_json_file = "data2text/iepile-ner-test-v2.json"
test_jsonl_file = "data2text/iepile-ner-test-v2.jsonl"


with open(test_json_file, "r", encoding='utf-8') as file:
    with jsonlines.open(test_jsonl_file, 'w') as writer:
        input_data = json.load(file)
        for data in input_data:
            writer.write(data)

rename_file("data2text/iepile-ner-test-v2.jsonl", "data2text/iepile-ner-test-json-v2.json")


File renamed from data2text/iepile-ner-test-v2.jsonl to data2text/iepile-ner-test-json-v2.json


## 生成测试集（按照单个schema）
1. 按照单个schema生成测试集，目前其他测试方法优先级更高，此处代码保留但不作运行

In [7]:
input_file_path_test = 'NERsample_test.json'

with open(input_file_path_test, 'r', encoding='utf-8') as file:
    input_data_test = json.load(file)

# 获取当前工作目录
current_path = os.getcwd()
output_file_path = os.path.join(current_path, 'schema_NERsample_test.json')

output_data_test = []
# 遍历输入数据中的每个data项
for entity_type in ["时间", "人物", "国家", "装备名称", "装备类型", "装备数量", "组织机构", "地点", "任务", "行动", "事件"]:
    output_file_path = os.path.join(current_path, entity_type + 'NERsample_test.json')
    for data in input_data_test:
        entities = []
        for item in data['entity']:
            if item["entity_type"] == entity_type:
                entities.append(item)
                output = {"text": data["text"], "entity": entities}
                output_data_test.append(output)
    # 打开输出路径
    with open(output_file_path, 'w', encoding='utf-8') as file:
        json.dump(output_data_test, file, ensure_ascii=False,indent=1)


In [29]:
input_file_path_test = 'NERsample_test.json'

with open(input_file_path_test, 'r', encoding='utf-8') as file:
    input_data_test = json.load(file)

# 获取当前工作目录
current_path = os.getcwd()

output_data_test = []
# 遍历输入数据中的每个data项

output_file_path = os.path.join(current_path,  '装备数量NERsample_test.json')
for data in input_data_test:
    for item in data['entity']:
        if item["entity_type"] == "装备数量":
            entities = []
            entities.append(item)
            output = {"text": data["text"], "entity": entities}
            output_data_test.append(output)
# 打开输出路径
with open(output_file_path, 'w', encoding='utf-8') as file:
    json.dump(output_data_test, file, ensure_ascii=False,indent=1)

In [10]:
def find_all_spans(source_text, substring):
    length = len(substring)
    spans = []
    for i in range(len(source_text) - length + 1):
        if source_text[i:i+length] == substring:
            spans.append((i, i + length))
    return spans


source_text = "这是一个测试文本，这包含一些用于测试的子字符串。"
substring = "字符串"

spans = find_all_spans(source_text, substring)
print(spans)
# span = find_span(source_text, substring)
# print(span)  # 输出子字符串的位置span

[(20, 23)]


### 将训练数据转换为模型微调使用的数据格式
运行格式转换脚本进行格式转换，其中指令含义如下：
- --src_path data2text/iepile-ner-train-val-json.json
   - 训练数据的路径
- --tgt_path data2text/iepile-ner-train-val-transformed.json
   - 数据转换之后文件输出的路径
- --language zh
   - 选择zh语言
- --task NER
   - 选择实体抽取任务
- --split_num 4
   - 定义单个指令中可包含的最大schema数目
- --random_sort
   - 是否对指令中的schema随机排序，默认为false
- --split train
   - 指定数据集类型为train

In [14]:
!python ie2instruction/convert_func.py \
    --src_path data2text/data2text-ner-train-val-json.json \
    --tgt_path data2text/data2text-ner-train-val-transformed.json \
    --schema_path data2text/schema.json \
    --language zh \
    --task NER \
    --split_num 4 \
    --random_sort \
    --split train

### 拆分训练集和验证集

In [15]:
import json
import random


# 读取JSONL文件
def read_jsonl(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        data = [json.loads(line) for line in file]
    return data


# 划分数据集
def split_data(data, train_ratio=0.8):
    random.shuffle(data)  # 随机打乱数据
    train_size = int(len(data) * train_ratio)
    train_data = data[:train_size]
    val_data = data[train_size:]
    return train_data, val_data


# 写入JSONL文件
def write_jsonl(data, file_path):
    with open(file_path, 'w', encoding='utf-8') as file:
        for entry in data:
            file.write(json.dumps(entry, ensure_ascii=False) + '\n')


# 主函数
def main(input_file, train_file, val_file):
    data = read_jsonl(input_file)
    train_data, val_data = split_data(data)
    write_jsonl(train_data, train_file)
    write_jsonl(val_data, val_file)
    print(f"训练集和验证集已生成：\n训练集大小：{len(train_data)}\n验证集大小：{len(val_data)}")


# 使用示例
input_file = 'data2text/data2text-ner-train-val-transformed.json'  # 输入的JSONL文件
train_file = 'data2text/data2text-ner-train-transformed.json'  # 训练集文件
val_file = 'data2text/data2text-ner-val-transformed.json'      # 验证集文件

main(input_file, train_file, val_file)


训练集和验证集已生成：
训练集大小：7732
验证集大小：1933


### 将测试数据转换为测试使用的数据格式
运行格式转换脚本进行格式转换，其中指令含义如下：
- --src_path data2text/iepile-ner-test-json.json
   - 训练数据的路径
- --tgt_path data2text/iepile-ner-test-transformed.json
   - 数据转换之后文件输出的路径
- --language zh
   - 选择zh语言
- --task NER
   - 选择实体抽取任务
- --split_num 4
   - 定义单个指令中可包含的最大schema数目
- --random_sort
   - 是否对指令中的schema随机排序，默认为false
- --split test
   - 指定数据集类型为train

In [16]:
!python ie2instruction/convert_func.py \
    --src_path data2text/data2text-ner-test-json.json \
    --tgt_path data2text/data2text-ner-test-transformed.json \
    --schema_path data2text/schema_test.json \
    --language zh \
    --task NER \
    --split_num 4 \
    --split test

## 将测试输出每6条合并
测试集的输出因为split_num = 4，故每条文本被拆分为6条数据输出，我们需要将其合并

In [5]:
import json


def merge_dicts(dict_list):
    """合并多个字典，将相同键的值合并为一个列表"""
    merged_dict = {}
    for d in dict_list:
        for key, value in d.items():
            if key in merged_dict:
                merged_dict[key].extend(value)
            else:
                merged_dict[key] = value
    return merged_dict


def process_jsonl_file(input_file, output_file):
    with open(input_file, 'r', encoding='utf-8') as f:
        lines = f.readlines()

    result = []
    temp_dict_list = []

    for i, line in enumerate(lines):
        data = json.loads(line)
        output_data = json.loads(data['output'])
        temp_dict_list.append(output_data)

        # 每6条数据合并一次
        if (i + 1) % 6 == 0 or i == len(lines) - 1:
            merged_output = merge_dicts(temp_dict_list)
            result.append({"result": merged_output})
            temp_dict_list = []

    # 将结果写入JSON文件
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(result, f, ensure_ascii=False, indent=4)


# 使用示例
input_file = 'results/baichuan7B-ft-test-output.json'
output_file = 'results/baichuan7B-ft-test-output-merged.json'

process_jsonl_file(input_file, output_file)


In [5]:
with open("config_train.json", "r") as file:
    config = json.load(file)
base_model = config.get("base_model", "cache/models--baichuan-inc--Baichuan2-7B-Chat/snapshots/ea66ced17780ca3db39bc9f8aa601d8463db3da5")
print(base_model)

cache/models--baichuan-inc--Baichuan2-7B-Chat/snapshots/ea66ced17780ca3db39bc9f8aa601d8463db3da5


In [6]:
import os
current_path = "data2text"
output_file_path = os.path.join(current_path, 'data2text-ner-train-val.json')
test_file_path = os.path.join(current_path, 'data2text-ner-test.json')

train_json_file = output_file_path
train_jsonl_file = os.path.join(current_path, 'data2text-ner-train-val.jsonl')
test_json_file = test_file_path
test_jsonl_file = os.path.join(current_path, 'data2text-ner-test.jsonl')
print(output_file_path)
print(test_file_path)
print(train_jsonl_file)
print(test_jsonl_file)

data2text/data2text-ner-train-val.json
data2text/data2text-ner-test.json
data2text/data2text-ner-train-val.jsonl
data2text/data2text-ner-test.jsonl
